### 4D CT PPV Analysis

Calculates the **Positive Predictive Value (PPV)** for 4D CT scans:

- **PPV = True Positives / (True Positives + False Positives)**
- **True Positives:** Exact, Side, or Partial matches
- **False Positives:** Incorrect Localization

Outputs total localized scans, match counts, and PPV %.

In [ ]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')

In [7]:
# Filter to 4D CT scans that localized (Left or Right)
localized_4dct = preop_imaging_df[
    preop_imaging_df['4D CT Imaging Side'].isin(['Left', 'Right'])
].copy()

# Calculate total localized scans
total_localized_4dct = localized_4dct.shape[0]

# Calculate true positives (Exact, Side, Partial) and false positives (Incorrect Localization)
true_positives = localized_4dct['4D CT Scan Calculated Concordance'].isin(
    ['Yes (Exact)', 'Yes (Side)', 'Yes (Partial)']
).sum()

false_positives = (localized_4dct['4D CT Scan Calculated Concordance'] == 'Incorrect Localization').sum()

# Calculate PPV
ppv = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else None

In [9]:
# Create results summary dataframe
ppv_summary_df = pd.DataFrame({
    'Metric': [
        'Total localized 4D CT scans',
        'True Positives (Exact/Side/Partial)',
        'False Positives (Incorrect Localization)',
        'PPV'
    ],
    'Value': [
        total_localized_4dct,
        true_positives,
        false_positives,
        f"{ppv:.2%}" if ppv is not None else 'N/A'
    ]
})

# Display results
print("4D CT Positive Predictive Value (PPV) Summary")
display(ppv_summary_df)

4D CT Positive Predictive Value (PPV) Summary


,Metric,Value
0,Total localized 4D CT scans,171
1,True Positives (Exact/Side/Partial),146
2,False Positives (Incorrect Localization),25
3,PPV,85.38%
